# Imports

In [2]:
# misc
import os, time, sys
import math, random
from timeit import default_timer as timer

# load/save files
import requests
import zipfile
import csv, json, sqlite3
import joblib
import h5py

# plot
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from PIL import Image

# datascience libs
import numpy as np
import pandas as pd

try: # python
    path_ = os.path.join(os.path.dirname(os.path.abspath(__file__)), "..")
except NameError: # jupyter notebook
    path_ = os.path.dirname(os.getcwd())

# Load CSV into DataFrame

In [3]:
#df = pd.DataFrame(bbox, columns=['x1','y1','x2','y2'])
#df.to_csv(file_path, index=False)
path_datasets = os.path.join(path_, "datasets")
df_magasins = pd.read_csv(os.path.join(path_datasets, "magasins.csv"))
df_produits = pd.read_csv(os.path.join(path_datasets, "produits.csv"))
df_ventes = pd.read_csv(os.path.join(path_datasets, "ventes.csv"))


In [4]:
display(df_magasins.head(3))
display(df_produits.head(3))
display(df_ventes.head(3))

,ID Magasin,Ville,Nombre de salariés
0,1,Paris,10
1,2,Marseille,5
2,3,Lyon,8


,Nom,ID Référence produit,Prix,Stock
0,Produit A,REF001,49.99,100
1,Produit B,REF002,19.99,50
2,Produit C,REF003,29.99,75


,Date,ID Référence produit,Quantité,ID Magasin
0,2023-05-27,REF001,5,1
1,2023-05-28,REF002,3,2
2,2023-05-29,REF003,2,1


# Create DB

In [35]:

def create_table(name_table, path=""):
    path_table = os.path.join(path, name_table)
    if os.path.exists(path_table):
        return
    
    with sqlite3.connect(path_table) as conn:
        cursor = conn.cursor()
    
    # magasins
    cursor.execute(
        "CREATE TABLE magasins (id INTEGER PRIMARY KEY AUTOINCREMENT, nom_ville TEXT, nombre_salaries INT)")
    # produits
    cursor.execute(
        "CREATE TABLE produits (id INTEGER PRIMARY KEY AUTOINCREMENT, id_ref_produit TEXT, nom TEXT, prix FLOAT, stock INT)")
    # ventes
    cursor.execute(
        "CREATE TABLE ventes (id INTEGER PRIMARY KEY AUTOINCREMENT, date DATE, id_ref_produit TEXT, quantite INT, id_magasin INT)")

    conn.commit()
    conn.close()


create_table("table.sqlite", path_datasets)

# Add new items

In [42]:
def insert_items_to_magasin(name_table, path="", list_nom_villes=[], list_salaries=[]):
    path_table = os.path.join(path, name_table)
    if not os.path.exists(path_table):
        return
    
    with sqlite3.connect(path_table) as conn:
        cursor = conn.cursor()

    query = """
        INSERT INTO magasins (nom_ville, nombre_salaries)
        SELECT
            T.*
        FROM
            (SELECT '{}' nom_ville,  {} nombre_salaries) T
                LEFT JOIN
            magasins on magasins.nom_ville = T.nom_ville
        WHERE
            magasins.nom_ville is null
        """.format("Martigues",36)
    
    cursor.execute(query)    
    conn.commit()
    conn.close()
    
    
insert_items_to_magasin("table.sqlite", path_datasets, ['Paris', 'Marseille'], [10, 5])



# Show DB

In [43]:
def show_table(name_table, path=""):
    path_table = os.path.join(path, name_table)
    if not os.path.exists(path_table):
        return
    
    with sqlite3.connect(path_table) as conn:
        cursor = conn.cursor()
    
    cursor.execute("SELECT * FROM magasins")
    rows = cursor.fetchall()
    print("\nmagasins:")
    print(rows)
    
    cursor.execute("SELECT * FROM produits")
    rows = cursor.fetchall()
    print("\nproduits:")
    print(rows)
    
    cursor.execute("SELECT * FROM ventes")
    rows = cursor.fetchall()
    print("\nventes:")
    print(rows)
    
    conn.close()
    
show_table("table.sqlite", path_datasets)


magasins:
[(1, 'Martigues', 36)]

produits:
[]

ventes:
[]


# Analyse Exploratoire

In [20]:
print("\nmagasins:")
display(df_magasins)
print("\nproduits:")
display(df_produits)
print("\nventes:")
display(df_ventes)


magasins:


,ID Magasin,Ville,Nombre de salariés
0,1,Paris,10
1,2,Marseille,5
2,3,Lyon,8
3,4,Bordeaux,12
4,5,Lille,6
5,6,Nantes,7
6,7,Strasbourg,9



produits:


,Nom,ID Référence produit,Prix,Stock
0,Produit A,REF001,49.99,100
1,Produit B,REF002,19.99,50
2,Produit C,REF003,29.99,75
3,Produit D,REF004,79.99,120
4,Produit E,REF005,39.99,80



ventes:


,Date,ID Référence produit,Quantité,ID Magasin
0,2023-05-27,REF001,5,1
1,2023-05-28,REF002,3,2
2,2023-05-29,REF003,2,1
3,2023-05-30,REF004,4,3
4,2023-05-31,REF005,7,2
5,2023-06-01,REF001,3,4
6,2023-06-02,REF002,6,1
7,2023-06-03,REF003,1,5
8,2023-06-04,REF004,2,3
9,2023-06-05,REF005,5,6
